In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWINB", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWINB", "T"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################


# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 4
Maxiterations_turbine = 30

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True
# Расчет для работы с теплофикацией
Teplo = int(True)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

#####################Максимов#####################
PKM_zaryad = True
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))


gas_streams0.loc["GTU-KU":"GPK-out", "H"] = Enthalpies

############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

# Расчет всей ПГУ вместе


def calculate_CCGT(
    Iterations_KU_TU,
    Iterations_cotel,
    Iterations_turbine,
):

    # Расчет ГТУ
    Gas_turbine_res = Gas_turbine.calc(GTU_input)

    # Запись данных об электричестве
    electric.at["GTU", "N"] = Gas_turbine_res["N"]
    electric.at["GTU", "KPD"] = Gas_turbine_res["eff"]
    electric.at["DK", "N"] = Gas_turbine_res["Ndk"]

    # Запись данных о газе на выходе из ГТУ
    gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
    gas_streams.at["GTU-KU", "T"] = Gas_turbine_res["T"]
    gas_streams.at["GTU-KU", "G"] = Gas_turbine_res["G"]
    gas_streams.at["GTU-KU", "P"] = 0.1
    gas_streams.at["GTU-KU", "H"] = gas0.p_t(
        gas_streams.at["GTU-KU", "P"], gas_streams.at["GTU-KU", "T"]
    )["h"]
    Gas_turbine_composition = pd.read_excel(
        "input.xlsx", sheet_name="Gas_composition0", index_col=0
    )
    
    #####################Максимов#####################
    from PKM import PKM_all
    PKM = PKM_all.calc(PKM_zaryad, gas_streams,syngas_streams,water_streams,water_streams0,heaters)
    steamVD_to_turbine = PKM['steamVD_to_turbine']
    ##################################################

    # Состав газов при частичной нагрузке
    fractiongas = list(gas_streams.loc["GTU-PEVD", "N2":"Ar"])

    gas1 = prop.Materials_prop(
        gasmix,
        fractiongas,
        prop.REFPROP_h_s,
        prop.REFPROP_p_t,
        prop.REFPROP_p_h,
        prop.REFPROP_p_s,
        prop.REFPROP_p_q,
        prop.REFPROP_t_q,
        prop.REFPROP_p_rho,
        prop.REFPROP_s_q,
        RP=RP,
    )

    # Инициализаця KU+TU, она здесь потому что нжно менять состав газа на входе в КУ
    
    KU_and_TU = KU_TU.ku_tu(
        gas0,
        gas1,
        water,
        gas_streams0,
        gas_streams,
        water_streams0,
        water_streams,
        heaters,
        electric,
        "PEVD-DROSVD",
        "PPND-DROSND",
        "DROSVD-TURBVD",
        "DROSND-TURBND",
        Calcmethod,
        KPD_SP,
        KPD_KN,
        KPD_to,
        KPD_PN,
        steamVD_fraction_to_turbine,
        steamVD_to_turbine,
    )

    start_time = time.time()

    # Расчет КУ и ТУ
    KU_and_TU.calculate(
        Teplo,
        Calctolerance,
        Iterations_KU_TU,
        Iterations_cotel,
        Iterations_turbine,
    )

    print(f"fin КУ и ТУ:--- {round((time.time() - start_time), 1)} сек. ---")


calculate_CCGT(Maxiterations_KU_TU, Maxiterations_cotel, Maxiterations_turbine)

Относительная мощность ГТУ в ГТУ 1
PKM+
Fin котел-утилизатора:--- 5.61 сек. ---
dQ/Qsumm 0.04925985940234744
Fin турбоустановка:--- 0.05 сек. ---
Достигнуто максимальное количество итераций контура высокого давления
Fin котел-утилизатора:--- 9.71 сек. ---
dQ/Qsumm 0.0008721825029175692
Fin турбоустановка:--- 0.04 сек. ---
Fin котел-утилизатора:--- 1.83 сек. ---
dQ/Qsumm -0.0019298718153663071
Fin турбоустановка:--- 0.04 сек. ---
Fin котел-утилизатора:--- 0.99 сек. ---
dQ/Qsumm -0.004477169698847399
Fin турбоустановка:--- 0.04 сек. ---
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 1.7934872262892247e-14, Error_nd_G: 0.030385166505516013, Error_vd_G: 0.0
Fin котел-утилизатора:--- 1.48 сек. ---
dQ/Qsumm -0.0038883830772663236
Fin турбоустановка:--- 0.04 сек. ---
Pnd_it [0.72922]
Pvd_it [8.84044]
Время 1 итерации расчета КУ+ТУ:--- %s сек. --- 19.8
Достигнуто максимальное количество итераций контура высокого давления
Fin котел-утилизатора:--- 6.23 сек. ---
dQ/Qsu

In [2]:
# Вывод газовых потоков
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,15.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.1,0.1,958.869199,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
REF-SMESH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-PEVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PEVD-IVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IVD-EVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EVD-PPND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PPND-IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IND-GPK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GPK-out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Вывод водяных потоков
water_streams

,T,P,H,G,S,X
AIR,15.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,526.786037,8.708773,3457.701439,75.044136,NaN,NaN
IVD-PEVD,304.598546,9.158292,2740.292554,75.044136,NaN,NaN
EVD-IVD,299.355076,9.158292,1340.633688,75.044136,NaN,NaN
PEN-EVD,165.020300,9.158292,702.160078,75.044136,NaN,NaN
BND-PEN,163.468920,0.674729,690.563548,75.044136,NaN,NaN
PPND-DROSND,207.989857,0.668401,2864.649742,15.740318,NaN,NaN
IND-PPND,163.468920,0.674729,2761.194139,15.740318,NaN,NaN
GPK-IND,161.642149,0.674729,682.629548,90.784454,NaN,NaN
GPK-REC,161.642149,0.674729,682.629548,94.551833,NaN,NaN


In [4]:
electric

,Ni,N,KPDm,KPD
PEN,0.870252,0.844900,1.030005,0.807400
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,150.750600,NaN,33.348086
DK,NaN,6.269747,NaN,NaN
Turbine,59.440853,NaN,NaN,NaN
Tots1,38.423645,NaN,NaN,0.847886
Tots2,15.348963,NaN,NaN,0.670341
Tots3,4.999816,NaN,NaN,0.583099
Tots4,0.668429,NaN,NaN,0.289028


In [5]:
heaters

,Qw,Qg,KPD
PEVD,53834.947707,54378.735058,0.99
IVD,105035.031771,106095.991688,0.99
EVD,47912.759021,48396.726284,0.99
PPND,1628.248625,1644.695581,0.99
IND,33313.140456,33649.636825,0.99
GPK,40742.319479,41153.858060,0.99
SP2,88.005718,NaN,0.99
SP1,70.307374,NaN,0.99
OD,6.076744,NaN,0.99
Strans,130850.501911,NaN,NaN
